In [ ]:
%cd ../..
%pip install .
import time
# def placeObjs(car, numCars):
#     for i in range(numCars):
#         car = Car ahead of car by Range(4, 5)
#         leftCar = Car left of car by Normal(2, 0.1), facing roadDirection
#         rightCar = Car right of car by Normal(3, 0.1), facing Range(0, 10) deg relative to ego.heading
#     return leftCar, rightCar

# spawn_point = 207.26 @ 8.72
# ego = Car at spawn_point, with visible_distance 200

# leftCar, rightCar = placeObjs(ego, 2)
# require (distance to leftCar) < 200
# require (distance to rightCar) < 200

In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as t1
join cars as t2 on t1.cameraId = t2.cameraId
join Cameras as cam on t1.cameraId = Cameras.id
where
  getX(t1.traj, cam.timestamp) < getX(cam.egoTranslation, cam.timestamp) AND
  getX(t2.traj, cam.timestamp) > getX(cam.egoTranslation, cam.timestamp) AND
  facingRelative(t1.heading, road_direction(t1.centroid, cam.timestamp), cam.timestamp) = 0 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) >= 0 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) < 10 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 200 AND
  DISTANCE(cam.egoTranslation, t2.centroid, cam.timestamp) < 200
"""
"""
Need to implement:
  SQL function getX(tgeompoints, timestamptz)
  SQL function getX(geometry)
  SQL function road_direction(tgeompoint, timestamptz)
  SQL function road_direction(geometry)
  SQL function facingRelative(tfloat, real, timestamptz)
  SQL function facingRelative(real, real)
  SQL function distance(geometry, tgeompoint, timestamptz)
  SQL function distance(geometry, geometry)
"""

In [ ]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

# ### Query ###
from apperception.utils import F
car_world = world.filter(lambda obj: obj.object_type == 'vehicle.car')

def pred_1(obj1, obj2, cam):
    return (
        F.get_x(obj1.traj, cam.timestamp) < F.get_x(cam.ego, cam.timestamp) and
        F.get_x(obj2.traj, cam.timestamp) > F.get_x(cam.ego, cam.timestamp)
    )
world_1 = car_world.filter(pred_1)

In [ ]:
start = time.time()

key_1 = world_1.get_traj_key()

end = time.time()
print(format(end-start))

In [ ]:
def pred_2(obj1, obj2, cam):
    return (
        F.facing_relative(obj1, F.road_direction(obj1, cam.timestamp), cam.timestamp) == 0
#         F.facing_relative(obj2, cam.ego, cam.timestamp) >= 0 and
#         F.facing_relative(obj2, cam.ego, cam.timestamp) < 10
    )
world_2 = car_world.filter(pred_2)

In [ ]:
start = time.time()

key_2 = world_2.get_traj_key()

end = time.time()
print(format(end-start))

In [ ]:
def pred_3(obj1, obj2, cam):
    return (
        F.distance(cam.ego, obj1, cam.timestamp) < 200 and
        F.distance(cam.ego, obj2, cam.timestamp) < 200
    )
world_3 = car_world.filter(pred_3)

In [ ]:
start = time.time()

key_3 = world_3.get_traj_key()

end = time.time()
print(format(end-start))

In [ ]:
def pred(obj1, obj2, cam):
    return (
        F.get_x(obj1.traj, cam.timestamp) < F.get_x(cam.ego, cam.timestamp) and
        F.get_x(obj2.traj, cam.timestamp) > F.get_x(cam.ego, cam.timestamp) and
        F.facing_relative(obj1, F.road_direction(obj1, cam.timestamp), cam.timestamp) == 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) >= 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) < 10 and
        F.distance(cam.ego, obj1, cam.timestamp) < 200 and
        F.distance(cam.ego, obj2, cam.timestamp) < 200
    )
final_world = car_world.filter(pred)
final_world.get_traj_key